In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "..")))



In [2]:
%pip install google-cloud-bigquery
%pip install pandas
%pip install db-dtypes
%pip install pandas-gbq


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:

from google.cloud import bigquery
import pandas as pd

from utils.import_table import import_table
from utils.export import export_table


In [4]:
# Initialiser le client BigQuery
client = bigquery.Client.from_service_account_json('../../key.json')

In [5]:

locations = import_table(client, 'location', dataset_id="ensemble_de_donnees_public")
warehouses = import_table(client, 'warehouse', dataset_id="ensemble_de_donnees_public")
stores = import_table(client, 'store', dataset_id="ensemble_de_donnees_public")
suppliers = import_table(client, 'supplier', dataset_id="ensemble_de_donnees_public")

# Visualiser le DataFrame sous forme de tableau
display(locations.head())
display(warehouses.head())
display(stores.head())
display(suppliers.head())

/Users/antoine/miniconda3/envs/AI/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,id,capacity,address,datastream_metadata
0,95,8843,4365 Webb Branch Suite 778,{'uuid': '8b991aa3-eca8-4dde-9949-4fa601101100...
1,5,8945,432 Mcdonald Heights Suite 249,{'uuid': '8b991aa3-eca8-4dde-9949-4fa600010010...
2,44,1092,96205 Bowers Knoll Suite 077,{'uuid': '8b991aa3-eca8-4dde-9949-4fa600111001...
3,89,6279,8652 Terri Keys Apt. 988,{'uuid': '8b991aa3-eca8-4dde-9949-4fa601100110...
4,25,4571,63269 Elizabeth Crescent Apt. 718,{'uuid': '8b991aa3-eca8-4dde-9949-4fa600100110...


,id,location_id,name,datastream_metadata
0,2,90,4551 Collins Isle Suite 355,{'uuid': '3c5c5bf5-dead-45c8-821f-6b8011001111...
1,1,38,8340 Mitchell Grove Suite 075,{'uuid': '3c5c5bf5-dead-45c8-821f-6b8011001110...
2,3,2,569 Michael Gateway Apt. 819,{'uuid': '3c5c5bf5-dead-45c8-821f-6b8011010000...


,id,name,location_id,datastream_metadata
0,2,6335 Brown Locks Apt. 475,61,{'uuid': '3c5c5bf5-dead-45c8-821f-6b8010110001...
1,5,59320 Michael Tunnel Apt. 009,43,{'uuid': '3c5c5bf5-dead-45c8-821f-6b8010110100...
2,6,55663 Samantha Trace Apt. 119,8,{'uuid': '3c5c5bf5-dead-45c8-821f-6b8010110101...
3,8,148 Lopez Prairie Apt. 548,95,{'uuid': '3c5c5bf5-dead-45c8-821f-6b8010110111...
4,10,83218 Lauren Vista,57,{'uuid': '3c5c5bf5-dead-45c8-821f-6b8010111001...


,id,entity_id,location_id,name,datastream_metadata
0,14,14,98,Collins-Johnson,{'uuid': 'e80bd850-1d95-4acd-ba8f-88d410111010...
1,8,8,81,Smith PLC,{'uuid': 'e80bd850-1d95-4acd-ba8f-88d410110100...
2,4,4,91,Berger-Bailey,{'uuid': 'e80bd850-1d95-4acd-ba8f-88d410110000...
3,6,6,14,Flores-Daniels,{'uuid': 'e80bd850-1d95-4acd-ba8f-88d410110010...
4,2,2,24,"Mullins, Torres and Waters",{'uuid': 'e80bd850-1d95-4acd-ba8f-88d410101110...


In [6]:
# Join des tables Locations et Warehouses sur id (locations) location_id (warehouses)
locations_warehouses = pd.merge(locations, warehouses, left_on='id', right_on='location_id')
locations_warehouses['type'] = 'warehouse'
locations_warehouses = locations_warehouses.drop(columns=['id_y', 'location_id'])
locations_warehouses = locations_warehouses.rename(columns={'id_x': 'id'})
display(locations_warehouses.head())

locations_stores = pd.merge(locations, stores, left_on='id', right_on='location_id')
locations_stores['type'] = 'store'
locations_stores = locations_stores.drop(columns=['id_y', 'location_id'])
locations_stores = locations_stores.rename(columns={'id_x': 'id'})
display(locations_stores.head())

locations_suppliers = pd.merge(locations, suppliers, left_on='id', right_on='location_id')
locations_suppliers['type'] = 'supplier'
locations_suppliers = locations_suppliers.drop(columns=['id_y', 'location_id', 'entity_id'])
locations_suppliers = locations_suppliers.rename(columns={'id_x': 'id'})
display(locations_suppliers.head())

,id,capacity,address,datastream_metadata_x,name,datastream_metadata_y,type
0,2,5807,38167 Butler Greens Apt. 899,{'uuid': '8b991aa3-eca8-4dde-9949-4fa600001111...,569 Michael Gateway Apt. 819,{'uuid': '3c5c5bf5-dead-45c8-821f-6b8011010000...,warehouse
1,90,2604,801 Erica Haven,{'uuid': '8b991aa3-eca8-4dde-9949-4fa601100111...,4551 Collins Isle Suite 355,{'uuid': '3c5c5bf5-dead-45c8-821f-6b8011001111...,warehouse
2,38,5914,7358 Amanda Freeway Suite 295,{'uuid': '8b991aa3-eca8-4dde-9949-4fa600110011...,8340 Mitchell Grove Suite 075,{'uuid': '3c5c5bf5-dead-45c8-821f-6b8011001110...,warehouse


,id,capacity,address,datastream_metadata_x,name,datastream_metadata_y,type
0,95,8843,4365 Webb Branch Suite 778,{'uuid': '8b991aa3-eca8-4dde-9949-4fa601101100...,148 Lopez Prairie Apt. 548,{'uuid': '3c5c5bf5-dead-45c8-821f-6b8010110111...,store
1,44,1092,96205 Bowers Knoll Suite 077,{'uuid': '8b991aa3-eca8-4dde-9949-4fa600111001...,445 Aaron Hills Suite 790,{'uuid': '3c5c5bf5-dead-45c8-821f-6b8010111000...,store
2,43,9113,64539 Fitzpatrick Pines Apt. 261,{'uuid': '8b991aa3-eca8-4dde-9949-4fa600111000...,59320 Michael Tunnel Apt. 009,{'uuid': '3c5c5bf5-dead-45c8-821f-6b8010110100...,store
3,8,2871,9613 Short Ridge,{'uuid': '8b991aa3-eca8-4dde-9949-4fa600010101...,55663 Samantha Trace Apt. 119,{'uuid': '3c5c5bf5-dead-45c8-821f-6b8010110101...,store
4,56,8082,494 Megan Courts Apt. 201,{'uuid': '8b991aa3-eca8-4dde-9949-4fa601000101...,35112 Gutierrez Tunnel,{'uuid': '3c5c5bf5-dead-45c8-821f-6b8010110010...,store


,id,capacity,address,datastream_metadata_x,name,datastream_metadata_y,type
0,44,1092,96205 Bowers Knoll Suite 077,{'uuid': '8b991aa3-eca8-4dde-9949-4fa600111001...,"Luna, Lopez and Carter",{'uuid': 'e80bd850-1d95-4acd-ba8f-88d410111011...,supplier
1,89,6279,8652 Terri Keys Apt. 988,{'uuid': '8b991aa3-eca8-4dde-9949-4fa601100110...,"Arroyo, Dunn and Williams",{'uuid': 'e80bd850-1d95-4acd-ba8f-88d410110110...,supplier
2,98,6448,70808 Murphy Isle Apt. 768,{'uuid': '8b991aa3-eca8-4dde-9949-4fa601101111...,Collins-Johnson,{'uuid': 'e80bd850-1d95-4acd-ba8f-88d410111010...,supplier
3,48,3016,283 Stacie Square,{'uuid': '8b991aa3-eca8-4dde-9949-4fa600111101...,"Gregory, Smith and Morse",{'uuid': 'e80bd850-1d95-4acd-ba8f-88d410111001...,supplier
4,27,1358,3042 Craig Squares Apt. 142,{'uuid': '8b991aa3-eca8-4dde-9949-4fa600101000...,"Meyer, Strickland and Lyons",{'uuid': 'e80bd850-1d95-4acd-ba8f-88d410110011...,supplier


In [7]:
# concaténation des tables
locations_warehouses_stores = pd.concat([locations_warehouses, locations_stores])
locations_warehouses_stores_suppliers = pd.concat([locations_warehouses_stores, locations_suppliers])

display(locations_warehouses_stores_suppliers.head())

,id,capacity,address,datastream_metadata_x,name,datastream_metadata_y,type
0,2,5807,38167 Butler Greens Apt. 899,{'uuid': '8b991aa3-eca8-4dde-9949-4fa600001111...,569 Michael Gateway Apt. 819,{'uuid': '3c5c5bf5-dead-45c8-821f-6b8011010000...,warehouse
1,90,2604,801 Erica Haven,{'uuid': '8b991aa3-eca8-4dde-9949-4fa601100111...,4551 Collins Isle Suite 355,{'uuid': '3c5c5bf5-dead-45c8-821f-6b8011001111...,warehouse
2,38,5914,7358 Amanda Freeway Suite 295,{'uuid': '8b991aa3-eca8-4dde-9949-4fa600110011...,8340 Mitchell Grove Suite 075,{'uuid': '3c5c5bf5-dead-45c8-821f-6b8011001110...,warehouse
0,95,8843,4365 Webb Branch Suite 778,{'uuid': '8b991aa3-eca8-4dde-9949-4fa601101100...,148 Lopez Prairie Apt. 548,{'uuid': '3c5c5bf5-dead-45c8-821f-6b8010110111...,store
1,44,1092,96205 Bowers Knoll Suite 077,{'uuid': '8b991aa3-eca8-4dde-9949-4fa600111001...,445 Aaron Hills Suite 790,{'uuid': '3c5c5bf5-dead-45c8-821f-6b8010111000...,store


In [8]:
# Variables pour BigQuery
dataset_name = "warehouse"
table_name = "location"

export_table(client, locations_warehouses_stores_suppliers, dataset_name, table_name)

Dataset warehouse déjà existant.


100%|██████████| 1/1 [00:00<00:00, 18157.16it/s]
